

---


---


#  **3RD PLACE WINNING SOLUTION NOTEBOOK**

---



---


# Network Traffic Scenario Prediction Challenge by ITU
Can you predict when a network fault will occure?

**Private LB Score: 0.77731,  Public LB Score:  0.77783**

**Downloading competition datasets directly from my gdive (where it's already saved) into colaboratory:**

In [ ]:
!gdown --id 1tKc0c6--Rxxtp9oKzT_q9QgSKQsecAWt
!gdown --id 1Dce8zAuAp7qif4isBJrM4zQsbnV7t_30

# **Unzip the datasets:**

In [ ]:
!unzip -qq /content/Train_data.zip
!unzip -qq /content/Test_data.zip

# **Installing pytorch-lightning:**

In [ ]:
!pip install -qq pytorch-lightning --quiet

# **Libraries:**

In [ ]:
import os
import gc
import glob
import torch
import shutil
import inspect
import numpy as np
import pandas as pd
from torch import nn
from torch import optim
from scipy.stats import mode
import pytorch_lightning as pl
from torchmetrics import Accuracy
from torch.utils.data import DataLoader
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import KFold, train_test_split
from pytorch_lightning.callbacks.progress.tqdm_progress import TQDMProgressBar

# **Config:**

In [ ]:
max_epochs = 15
patience = 10
n_splits = 10
layer_n = 32
lr = 1e-3

pool_stride = 5
kernel_size = 7
depth = 3
block_depth = 2

dropout = 0.08

sub_name = "sub_epo15_bd2_ks7_do0.08_ps5_ln32_d3.csv"

# **Reading Datasets:**

In [ ]:
!ls

sample_data	      Test_data      Train_data      variableDefinition.txt
SampleSubmission.csv  Test_data.zip  Train_data.zip


In [ ]:
# Checking the list of train files provided
!ls '/content/Train_data'

Train0.csv   Train21.csv  Train33.csv  Train45.csv  Train57.csv  Train69.csv
Train10.csv  Train22.csv  Train34.csv  Train46.csv  Train58.csv  Train6.csv
Train11.csv  Train23.csv  Train35.csv  Train47.csv  Train59.csv  Train70.csv
Train12.csv  Train24.csv  Train36.csv  Train48.csv  Train5.csv	 Train71.csv
Train13.csv  Train25.csv  Train37.csv  Train49.csv  Train60.csv  Train72.csv
Train14.csv  Train26.csv  Train38.csv  Train4.csv   Train61.csv  Train73.csv
Train15.csv  Train27.csv  Train39.csv  Train50.csv  Train62.csv  Train74.csv
Train16.csv  Train28.csv  Train3.csv   Train51.csv  Train63.csv  Train75.csv
Train17.csv  Train29.csv  Train40.csv  Train52.csv  Train64.csv  Train76.csv
Train18.csv  Train2.csv   Train41.csv  Train53.csv  Train65.csv  Train77.csv
Train19.csv  Train30.csv  Train42.csv  Train54.csv  Train66.csv  Train7.csv
Train1.csv   Train31.csv  Train43.csv  Train55.csv  Train67.csv  Train8.csv
Train20.csv  Train32.csv  Train44.csv  Train56.csv  Train68.csv  Train9.csv


In [ ]:
# Checking the list of test files provided
!ls '/content/Test_data'

Test0.csv   Test12.csv	Test15.csv  Test18.csv	Test3.csv  Test6.csv  Test9.csv
Test10.csv  Test13.csv	Test16.csv  Test1.csv	Test4.csv  Test7.csv
Test11.csv  Test14.csv	Test17.csv  Test2.csv	Test5.csv  Test8.csv


In [ ]:
# Checking number of train files provided
all_train_files = glob.glob('Train_data/*.csv')
len(all_train_files)

78

In [ ]:
# Checking number of test files provided
all_test_files = glob.glob('Test_data/*.csv')
len(all_test_files)

19

# **Generating single Train dataset:**

In [ ]:
%%time
import pandas as pd
import glob
import os

all_train_files = glob.glob('Train_data/*.csv')

# Creating a list for the multiple files
files = list(all_train_files)

print("Resultant CSV after joining all Train data CSV files at a particular location...");

def get_train(file):
    df = pd.read_csv(file)
    x = np.log1p(df[['time', 'portPktIn', 'portPktOut', 'qSize']].T.values.astype('float32'))

    # Target
    y = df['label'].values
    return x, y

train = list(map(get_train, files))
len(train)

Resultant CSV after joining all Train data CSV files at a particular location...
CPU times: user 3.92 s, sys: 635 ms, total: 4.56 s
Wall time: 4.58 s


78

In [ ]:
x, y = train[0]

In [ ]:
x.shape, y.shape

((4, 110000), (110000,))

# **Generating single Test dataset:**

In [ ]:
import pandas as pd
import glob
import os

all_train_files = glob.glob('Test_data/*.csv')

# Ceating a list for the multiple files
files = list(all_test_files)

print("Resultant CSV after joining all Test data CSV files at a particular location...");

def get_test(file):
    df = pd.read_csv(file)
    x = np.log1p(df[['time', 'portPktIn', 'portPktOut', 'qSize']].T.values.astype('float32'))
    return x

# Joining files with concat and read_csv
test = list(map(get_test, files))
len(test)

Resultant CSV after joining all Test data CSV files at a particular location...


19

In [ ]:
sub_df = pd.read_csv('SampleSubmission.csv')
sub_df.head()

,ID,Target
0,test0_0,0
1,test0_1,0
2,test0_2,0
3,test0_3,0
4,test0_4,0


# **MODELLING:**

# **Model Architechture (structure) Source**:
- **Edited from:** https://www.kaggle.com/code/super13579/u-net-1d-cnn-with-pytorch

In [ ]:
class conbr_block(nn.Module):
    def __init__(self, in_layer, out_layer, kernel_size, stride, dilation):
        super(conbr_block, self).__init__()

        self.conv1 = nn.Conv1d(
            in_layer, out_layer, kernel_size=kernel_size,
            stride=stride, dilation = dilation, padding =kernel_size//2,
            bias=True)
        self.bn = nn.BatchNorm1d(out_layer)
        self.relu = nn.ReLU()

    def forward(self,x):
        x = self.conv1(x)
        x = self.bn(x)
        out = self.relu(x)

        return out

class se_block(nn.Module):
    def __init__(self,in_layer, out_layer):
        super(se_block, self).__init__()

        self.conv1 = nn.Conv1d(in_layer, out_layer//8, kernel_size=1, padding=0)
        self.conv2 = nn.Conv1d(out_layer//8, in_layer, kernel_size=1, padding=0)
        self.fc = nn.Linear(1,out_layer//8)
        self.fc2 = nn.Linear(out_layer//8,out_layer)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self,x):

        x_se = nn.functional.adaptive_avg_pool1d(x,1)
        x_se = self.conv1(x_se)
        x_se = self.relu(x_se)
        x_se = self.conv2(x_se)
        x_se = self.sigmoid(x_se)

        x_out = torch.add(x, x_se)
        return x_out

class re_block(nn.Module):
    def __init__(self, in_layer, out_layer, kernel_size, dilation):
        super(re_block, self).__init__()

        self.cbr1 = conbr_block(in_layer,out_layer, kernel_size, 1, dilation)
        self.cbr2 = conbr_block(out_layer,out_layer, kernel_size, 1, dilation)
        self.seblock = se_block(out_layer, out_layer)

    def forward(self,x):

        x_re = self.cbr1(x)
        x_re = self.cbr2(x_re)
        x_re = self.seblock(x_re)
        x_out = torch.add(x, x_re)
        return x_out

class Unet_1D(nn.Module):
    def __init__(self, input_n, class_n,layer_n, pool_stride=5, kernel_size=7, depth=3, dropout=0.08, block_depth=2):
        super().__init__()
        self.input_n = input_n
        self.layer_n = layer_n
        self.kernel_size = kernel_size
        self.depth = depth
        self.class_n = class_n

        self.dropout = nn.Dropout(p=dropout)

        self.down_pools = nn.ModuleList([
            nn.AvgPool1d(input_n, stride=pool_stride**(i+1), ceil_mode=True) for i in range(depth-1)
        ])

        self.layer0 = self.down_layer(self.input_n, self.layer_n, self.kernel_size,1, 2)
        self.layers = nn.ModuleList([])
        for k in range(depth):
            layer = self.down_layer(
                int(self.layer_n*(k+1)) + self.input_n*(k!=0),
                int(self.layer_n*(k+2)),
                self.kernel_size,
                pool_stride, block_depth
            )
            self.layers.append(layer)

        self.cbr_ups = nn.ModuleList([])
        for k in range(depth):
            layer = conbr_block(int(self.layer_n*(1+2*(k+1))), self.layer_n*(k+1), self.kernel_size, 1, 1)
            self.cbr_ups.insert(0,layer)

        self.upsample = nn.Upsample(scale_factor=pool_stride, mode='nearest')

        self.outcov = nn.Conv1d(self.layer_n, self.class_n,
                                kernel_size=self.kernel_size, stride=1,padding=kernel_size//2,
                               )

    def down_layer(self, input_layer, out_layer, kernel, stride, depth):
        block = []
        block.append(conbr_block(input_layer, out_layer, kernel, stride, 1))
        for i in range(depth):
            block.append(re_block(out_layer,out_layer,kernel,1))
        return nn.Sequential(*block)

    def forward(self, x):

        pools = [down_pool(x) for down_pool in self.down_pools]

        #############Encoder#####################
        x = self.layer0(x)
        outs = [x]
        for k, layer in enumerate(self.layers):
            x = layer(x)
            if k < (len(self.layers) - 1):
                outs.insert(0, x)
                x = x[..., :pools[k].shape[-1]].contiguous()
                x = torch.cat([x, pools[k]],1)

        #############Decoder####################
        up = x
        for out, cbr_up in zip(outs, self.cbr_ups):
            up = self.upsample(up)
            up = up[..., :out.shape[-1]].contiguous()
            up = torch.cat([up, out], dim=1)
            up = cbr_up(up)

        ### dropout first
        up = self.dropout(up)

        out = self.outcov(up)

        return out

In [ ]:
len(train), len(test)

(78, 19)

In [ ]:
class TrafficModel(pl.LightningModule):
    def __init__(self, input_n=4, class_n=12, layer_n=32, pool_stride=5, kernel_size=7, depth=3, dropout=0.08, block_depth=2, lr=1e-3):
        super().__init__()
        self.save_hyperparameters()

        self.model = Unet_1D(input_n, class_n, layer_n, pool_stride, kernel_size, depth, dropout, block_depth)

        self.compute_loss = nn.CrossEntropyLoss()
        self.compute_acc = Accuracy(task='multiclass', num_classes=class_n)

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx=0):
        x, label = batch

        y = self(x)
        loss = self.compute_loss(y, label)
        acc = self.compute_acc(y, label)

        self.log('loss', loss, prog_bar=True, on_step=True, on_epoch=True)
        self.log('acc', acc, prog_bar=True, on_step=False, on_epoch=True)

        return loss

    def validation_step(self, batch, batch_idx=0):
        x, label = batch

        y = self(x)
        loss = self.compute_loss(y, label)
        acc = self.compute_acc(y, label)

        self.log('val_loss', loss, prog_bar=True, on_step=False, on_epoch=True)
        self.log('val_acc', acc, prog_bar=True, on_step=False, on_epoch=True)

    def predict_step(self, batch, batch_idx=0):
        x = batch
        y = self(x).softmax(dim=1)
        return {'Target': y.cpu().numpy()}

    def configure_optimizers(self):
        optimizer = optim.AdamW(self.parameters(), lr=self.hparams.lr, weight_decay=0.1)
        scheduler = optim.lr_scheduler.OneCycleLR(
            optimizer,
            max_lr=self.hparams.lr,
            total_steps=self.trainer.estimated_stepping_batches,
            pct_start=0.1,
        )

        scheduler = {
            'scheduler': scheduler,
            'interval': 'step'
        }
        return {'optimizer': optimizer, 'lr_scheduler': scheduler}

# **Setting up Callback:**

In [ ]:
def remove_dir(path):
    try:
        shutil.rmtree(path)
    except:
        pass

def free_memory(to_delete: list):
    calling_namespace = inspect.currentframe().f_back

    for _var in to_delete:
        calling_namespace.f_locals.pop(_var, None)
        gc.collect()
        torch.cuda.empty_cache()

def get_callbacks(fold=0):

    checkpoint_callback = pl.callbacks.ModelCheckpoint(
          filename=f"fold{fold}-" + "{epoch}-{val_loss:0.4f}-{val_acc:0.4f}",
          monitor='val_loss',
          verbose=False,
          save_last=False,
          save_top_k=1,
          mode='min',
          save_weights_only=True
    )

    early_stop_callback = pl.callbacks.EarlyStopping(
        monitor="val_loss",
        patience=patience,
        verbose=True,
        mode='min',
        strict=True,
        check_finite=True,
        check_on_train_epoch_end=False
    )
    prog_rate = TQDMProgressBar(refresh_rate=1)

    return [
        checkpoint_callback,
        early_stop_callback,
        prog_rate,
    ]

In [ ]:
test_loader = DataLoader(test, batch_size=1, shuffle=False)

In [ ]:
torch.set_float32_matmul_precision('high')

# **Model Training Starts here !!!**

In [ ]:
df_preds = []
fold_scores = []

kf = KFold(
    n_splits=n_splits,
    random_state=42,
    shuffle=True,
)

checkpoint_paths = []

for fold, (tr_idx, val_idx) in enumerate(kf.split(train)):
    print(f"\n#####Starting fold {fold}#####\n")

    # Make reproducible
    pl.seed_everything(42, workers=True)

    # Call the model here
    model = TrafficModel(
        layer_n=layer_n,
        pool_stride=pool_stride,
        kernel_size=kernel_size,
        depth=depth,
        dropout=dropout,
        block_depth=block_depth,
        lr=lr,
    )

    # Dataloaders
    train_dataset = [data_point for data_index, data_point in enumerate(train) if data_index in tr_idx]
    val_dataset = [data_point for data_index, data_point in enumerate(train) if data_index in val_idx]

    train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True, num_workers=2)
    val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False, num_workers=2)

    callbacks = get_callbacks(fold=fold)

    trainer = pl.Trainer(
        callbacks=callbacks,
        accelerator='gpu',
        devices=1,
        num_sanity_val_steps=1,
        max_epochs=max_epochs,
        logger=None,
    )

    trainer.fit(model, train_dataloaders=train_loader, val_dataloaders=val_loader)

    # Get prediction
    preds = trainer.predict(ckpt_path='best', dataloaders=test_loader)
    df_pred = pd.DataFrame(preds)
    df_preds.append(df_pred)

    # Include the paths in the list
    checkpoint_path = trainer.checkpoint_callback.best_model_path
    score = trainer.early_stopping_callback.best_score

    checkpoint_paths.append(checkpoint_path)
    fold_scores.append(score)

    free_memory([model, trainer])
    print(checkpoint_path)


INFO:lightning_fabric.utilities.seed:Global seed set to 42



#####Starting fold 0#####



INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loading `train_dataloader` to estimate number of stepping batches.
INFO:pytorch_lightning.callbacks.model_summary:
  | Name         | Type               | Params
----------------------------------------------------
0 | model        | Unet_1D            | 1.3 M 
1 | compute_loss | CrossEntropyLoss   | 0     
2 | compute_acc  | MulticlassAccuracy | 0     
----------------------------------------------------
1.3 M     Trainable params
0         Non-trainable params
1.3 M     Total params
5.148     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=15` reached.
INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at /content/lightning_logs/version_9/checkpoints/fold9-epoch=10-val_loss=0.5688-val_acc=0.7688.ckpt
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from the checkpoint at /content/lightning_logs/version_9/checkpoints/fold9-epoch=10-val_loss=0.5688-val_acc=0.7688.ckpt


Predicting: 0it [00:00, ?it/s]

/content/lightning_logs/version_9/checkpoints/fold9-epoch=10-val_loss=0.5688-val_acc=0.7688.ckpt


In [ ]:
checkpoint_paths

['/content/lightning_logs/version_0/checkpoints/fold0-epoch=14-val_loss=0.5223-val_acc=0.7916.ckpt',
 '/content/lightning_logs/version_1/checkpoints/fold1-epoch=12-val_loss=0.5315-val_acc=0.7817.ckpt',
 '/content/lightning_logs/version_2/checkpoints/fold2-epoch=14-val_loss=0.5384-val_acc=0.7790.ckpt',
 '/content/lightning_logs/version_3/checkpoints/fold3-epoch=14-val_loss=0.5513-val_acc=0.7677.ckpt',
 '/content/lightning_logs/version_4/checkpoints/fold4-epoch=13-val_loss=0.5532-val_acc=0.7618.ckpt',
 '/content/lightning_logs/version_5/checkpoints/fold5-epoch=13-val_loss=0.5125-val_acc=0.7930.ckpt',
 '/content/lightning_logs/version_6/checkpoints/fold6-epoch=12-val_loss=0.5315-val_acc=0.7738.ckpt',
 '/content/lightning_logs/version_7/checkpoints/fold7-epoch=14-val_loss=0.5885-val_acc=0.7657.ckpt',
 '/content/lightning_logs/version_8/checkpoints/fold8-epoch=12-val_loss=0.5812-val_acc=0.7532.ckpt',
 '/content/lightning_logs/version_9/checkpoints/fold9-epoch=10-val_loss=0.5688-val_acc=0.76

In [ ]:
print(torch.stack(fold_scores).mean())

tensor(0.5479, device='cuda:0')


# **Making Prediction on Test datsest:**

In [ ]:
np.mean(pd.concat(df_preds, axis=1).iloc[0].tolist(), axis=0).argmax(axis=1).flatten()

array([4, 4, 4, ..., 5, 5, 5])

In [ ]:
df_pred = pd.concat(df_preds, axis=1).apply(lambda row: np.array(row.tolist()).mean(axis=0).argmax(axis=1).flatten(), axis=1).rename('Target').to_frame()

In [ ]:
df_pred.head()

,Target
0,"[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, ..."
1,"[9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 10, 10, 10, 11,..."
2,"[8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, ..."
3,"[9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, ..."
4,"[8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, ..."


# **Submission:**

In [ ]:
sub_df = pd.read_csv('SampleSubmission.csv')

df_pred['file_id'] = [os.path.basename(t).split('.')[0].lower() for t in all_test_files]

df_pred['time'] = df_pred['Target'].apply(lambda t: np.arange(len(t)))
df_pred = df_pred.explode(['Target', 'time'])

df_pred['ID'] = df_pred.apply(lambda row: f"{row['file_id']}_{row['time']}", axis=1)
sub = df_pred[['ID', 'Target']].reset_index(drop=True)
sub = sub.set_index('ID').loc[sub_df['ID'].tolist()].reset_index()

sub.head()

,ID,Target
0,test0_0,8
1,test0_1,9
2,test0_2,9
3,test0_3,9
4,test0_4,9


In [ ]:
sub.to_csv(sub_name, index=False)

In [ ]:
sub['Target'].value_counts()

9     251151
11    226719
5     223762
6     203967
7     199867
3     199437
4     187806
0     185994
1     170681
8     170277
2     169856
10    148483
Name: Target, dtype: int64

In [ ]:
from google.colab import files
files.download(sub_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>